# Inset Map of Kotlin Island

Kotlin island is situated in Gulf of Finland and is one of districts of the city of Saint Petersburg in Russia.

This example shows how Lets-Plot-Kotlin SpatialDataset integration can help to build an inset map of Kotlin island.

[The geodata is provided by © OpenStreetMap contributors and is made available here under the Open Database License (ODbL)](https://www.openstreetmap.org/copyright).

In [1]:
%useLatestDescriptors
%use klaxon
%use lets-plot

In [2]:
%use lets-plot-gt(gt="[30,)")

In [3]:
@file:DependsOn("org.geotools:gt-main:[30,)")
@file:DependsOn("org.geotools:gt-geojson:[30,)")
@file:DependsOn("org.geotools:gt-shapefile:[30,)")
@file:DependsOn("org.geotools:gt-cql:[30,)")

In [4]:
import java.io.File
import java.io.ByteArrayInputStream
import java.io.InputStream
import java.net.URL

import org.geotools.data.shapefile.ShapefileDataStoreFactory
import org.geotools.data.simple.SimpleFeatureCollection
import org.geotools.api.data.DataStore
import org.geotools.api.data.DataStoreFinder
import org.geotools.feature.DefaultFeatureCollection
import org.geotools.feature.simple.SimpleFeatureTypeBuilder
import org.geotools.api.feature.simple.SimpleFeature
import org.geotools.filter.text.cql2.CQL
import org.geotools.geojson.feature.FeatureJSON

In [5]:
val f = File("../../../data/kotlin_places.geojson");
f.readText()

{
"type": "FeatureCollection",
"features": [
{ "type": "Feature", "properties": { "name": "Kronstadt Naval Cathedral", "type": "church" }, "geometry": { "type": "Point", "coordinates": [ 29.777455, 59.991744 ] } },
{ "type": "Feature", "properties": { "name": "Kronstadt History Museum", "type": "museum" }, "geometry": { "type": "Point", "coordinates": [ 29.791317, 59.986777 ] } },
{ "type": "Feature", "properties": { "name": "Kronstadt Naval Museum", "type": "museum" }, "geometry": { "type": "Point", "coordinates": [ 29.763422, 59.996108 ] } },
{ "type": "Feature", "properties": { "name": "City Russian Cemetery", "type": "cemetery" }, "geometry": { "type": "Point", "coordinates": [ 29.70613, 60.019788 ] } },
{ "type": "Feature", "properties": { "name": "Kronstadt Lutheran Cemetery", "type": "cemetery" }, "geometry": { "type": "Point", "coordinates": [ 29.749861, 60.002212 ] } },
{ "type": "Feature", "properties": { "name": "Vladimir Church", "type": "church" }, "geometry": { "type": "P

In [6]:
val params1: MutableMap<String, Any> = HashMap()
params1["url"] = f.toURI().toURL()
println(DataStoreFinder.getDataStore(params1))

val params2: MutableMap<String, Any> = HashMap()
params2["inputStream"] = ByteArrayInputStream(f.readText().toByteArray())
println(DataStoreFinder.getDataStore(params2))

val params3: MutableMap<String, Any> = HashMap()
params3["file"] = f
println(DataStoreFinder.getDataStore(params3))

null
null
null


In [7]:
val featureJSON = FeatureJSON()
featureJSON

org.geotools.geojson.feature.FeatureJSON@2b659e2d

In [8]:
val featureCollection = featureJSON.readFeatureCollection(f.reader())
println(featureCollection)
println(featureCollection.schema)

org.geotools.feature.DefaultFeatureCollection@729311c1
SimpleFeatureTypeImpl http://geotools.org:feature identified extends Feature(name:name,type:type,geometry:geometry)


In [9]:
val simpleFeatureCollection = DefaultFeatureCollection()
val features = featureCollection.features()
while (features.hasNext()) {
    val feature = features.next()
    if (feature is SimpleFeature) {
        simpleFeatureCollection.add(feature as SimpleFeature)
    }
}
simpleFeatureCollection.toSpatialDataset()

null
java.lang.NullPointerException
	at org.jetbrains.letsPlot.toolkit.geotools.FeatureCollectionExtKt.getDataAndGeometries(FeatureCollectionExt.kt:57)
	at org.jetbrains.letsPlot.toolkit.geotools.FeatureCollectionExtKt.toSpatialDataset(FeatureCollectionExt.kt:21)
	at org.jetbrains.letsPlot.toolkit.geotools.FeatureCollectionExtKt.toSpatialDataset$default(FeatureCollectionExt.kt:19)
	at Line_16_jupyter.<init>(Line_16.jupyter.kts:9)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:105)
	at kotlin.script.experimen

In [10]:
import java.net.URL

In [11]:
data class BBox(val xmin: Double, val ymin: Double, val xmax: Double, val ymax: Double)
class Geometry(val type: String, val coordinates: List<Any>)
class FeatureCollection(val type: String, val properties: Map<String, Any>, val geometry: Geometry)
class GeoJSON(val type: String, val features: List<FeatureCollection>)

In [12]:
fun getSpatialData(geoJSONData: String): SpatialDataset {
    val parsedGeoJson: GeoJSON = Klaxon().parse<GeoJSON>(geoJSONData) ?: throw Exception("Parsing error")
    val geometries = parsedGeoJson.features.map { feature -> Klaxon().toJsonString(feature.geometry) }
    val properties = parsedGeoJson.features.map { feature -> feature.properties }
    val propertiesData = if (properties.isEmpty()) {
        emptyMap()
    } else {
        properties[0].keys.map { col: String -> col to properties.map { record -> record[col] } }.toMap()
    }
    return SpatialDataset.withGEOJSON(data = propertiesData, geometry = geometries)
}

Line_19.jupyter.kts (1:42 - 56) Unresolved reference: SpatialDataset
Line_19.jupyter.kts (10:12 - 26) Unresolved reference: SpatialDataset

In [13]:
val kotlinBBox = BBox(29.63, 59.965, 29.815, 60.035)

#### Load boundaries of St.-Petersburg districts.

In [14]:
val spbDistrictsURL = "https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/spb_districts.geojson"
val spbDistrictsData = URL(spbDistrictsURL).readText()
val spbDistrictsSpatialData = getSpatialData(spbDistrictsData)

Line_21.jupyter.kts (3:31 - 45) Unresolved reference: getSpatialData

#### Create a map showing all districts of St.-Petersburg.

This map will become the **inset map**. The red rectangle indicates the bounds of the future **main map**.

In [15]:
val spbPlot = letsPlot() +
    geomPolygon(map = spbDistrictsSpatialData, color = "#a1d99b", fill = "#f7fcf5") +
    geomRect(xmin = kotlinBBox.xmin, ymin = kotlinBBox.ymin, xmax = kotlinBBox.xmax, ymax = kotlinBBox.ymax, color = "red", alpha = 0) +
    geomText(label = "Saint Petersburg", x = 30.334445, y = 59.934294, color = "black", size = 6) +
    scaleXContinuous(expand = listOf(0.0, 0.0)) + scaleYContinuous(expand = listOf(0.0, 0.0)) +
    themeVoid() + theme(panelBackground = elementRect(color = "black", fill = "white"))
spbPlot

Line_22.jupyter.kts (2:23 - 46) Unresolved reference: spbDistrictsSpatialData

#### Create the main map with only Kotlin island on it.

We use `xlim` and `ylim` parameters of the coordinate system to crop the entire map containing all districts of St.-Petersburg.

In [16]:
// SpatialDataset containing names and coordinates of some tourist attractions to show on the main map
val kotlinPlacesURL = "https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/kotlin_places.geojson"
val kotlinPlacesData = URL(kotlinPlacesURL).readText()
val kotlinPlacesSpatialData = getSpatialData(kotlinPlacesData)

Line_23.jupyter.kts (4:31 - 45) Unresolved reference: getSpatialData

In [17]:
// Cut-out the Kotlin area and add layes with text and points of interest.
val kotlinPlot = letsPlot() +
    geomRect(xmin = kotlinBBox.xmin, ymin = kotlinBBox.ymin, xmax = kotlinBBox.xmax, ymax = kotlinBBox.ymax, fill = "#aadaff", alpha = 0.2) +
    geomPolygon(map = spbDistrictsSpatialData, color = "#31a354", fill = "#e5f5e0") +
    geomPoint(map = kotlinPlacesSpatialData, size = 5) { color = "type"; shape = "type" } +
    geomText(map = kotlinPlacesSpatialData, hjust = "right", position = positionNudge(x = -.002)) { label = "name" } +
    geomText(label = "Kotlin Isl.", x = 29.725, y = 60.011, color = "#31a354", size = 13, fontface = "italic") +
    geomText(label = "Gulf of Finland", x = 29.665, y = 60.002, color = "#578bcc", size = 11, fontface = "italic") +
    coordCartesian(xlim = Pair(kotlinBBox.xmin, kotlinBBox.xmax), ylim = Pair(kotlinBBox.ymin, kotlinBBox.ymax)) +
    ggtitle("Tourist attractions on Kotlin island") +
    themeVoid() + theme().legendPosition(.15, .2)
kotlinPlot

Line_24.jupyter.kts (4:23 - 46) Unresolved reference: spbDistrictsSpatialData
Line_24.jupyter.kts (5:21 - 44) Unresolved reference: kotlinPlacesSpatialData
Line_24.jupyter.kts (6:20 - 43) Unresolved reference: kotlinPlacesSpatialData

#### Finally, use `GGBunch` to show these two maps together.

In [18]:
val bunch = GGBunch()
bunch.addPlot(kotlinPlot, 0, 0, 800, 600)
bunch.addPlot(spbPlot, 600, 25, 200, 150)
bunch.show()

Line_25.jupyter.kts (2:15 - 25) Unresolved reference: kotlinPlot
Line_25.jupyter.kts (3:15 - 22) Unresolved reference: spbPlot